* Author: Gordon Erlebacher
* Date: 2022-03-03
* Purpose: convert Sent field to either date (nb seconds from a fixed time) or an empty field.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import date_library as datelib

In [ ]:
filenm = "output_with_stats_columns.csv.gz"
filenm = "output_0211.csv"
df = pd.read_csv(filenm)

In [ ]:
date = datelib.DateClass()
sent_lst = df['Sent'].values
date.normalize_dates(sent_lst)

In [ ]:
date.create_dates_file(output_file="dates2.csv")

In [ ]:
date.update_output_file(df, "output_03112022_dates.csv")

In [ ]:
df.columns